In [ ]:
#Beginning project in which I will clean and analyze aviation data for the purpose of making 3 recommendations regarding mitigating risk when entering the aircraft industry. My analysis will seek to determine which specific characteristics of aircraft are associated with increased safety (certain engine types, amounts of engines, models, etc) Anyone entering into the aircraft industry and seeking to purchase planes could utilize this information in order to minimize risk in their investment by purchasing planes that are equipped with features associated with increased safety or by purchasing planes that are themselves safer. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
#importing pandas,numpy,seaborn,matplotlib

path = "C:\\Users\\agamb\\Documents\\Flatiron\\Phase1\\Aviation_Data.csv"
df = pd.read_csv(path)
df.head()
#importing the data using pandas and displaying the head

C:\Users\agamb\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [2]:
df.info()
#investigating data's columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [3]:
df.isnull().sum()
#checking for total null rows per column

Event.Id                   1459
Investigation.Type            0
Accident.Number            1459
Event.Date                 1459
Location                   1511
Country                    1685
Latitude                  55966
Longitude                 55975
Airport.Code              40099
Airport.Name              37558
Injury.Severity            2459
Aircraft.damage            4653
Aircraft.Category         58061
Registration.Number        2776
Make                       1522
Model                      1551
Amateur.Built              1561
Number.of.Engines          7543
Engine.Type                8536
FAR.Description           58325
Schedule                  77766
Purpose.of.flight          7651
Air.carrier               73700
Total.Fatal.Injuries      12860
Total.Serious.Injuries    13969
Total.Minor.Injuries      13392
Total.Uninjured            7371
Weather.Condition          5951
Broad.phase.of.flight     28624
Report.Status              7840
Publication.Date          16689
dtype: i

In [ ]:
#cleaning make and model

In [4]:
df = df.dropna(subset=['Make', 'Model'], how='any')
#deleting rows that are null for either Make or Model

In [5]:
df = df.drop(columns=['Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'FAR.Description', 'Schedule', 'Air.carrier'])
#dropping irrelevant columns

In [6]:
df["Injury.Severity"].value_counts()
#checking the value counts of injury severity

Non-Fatal     67305
Fatal(1)       6159
Fatal          5248
Fatal(2)       3703
Incident       2214
              ...  
Fatal(72)         1
Fatal(153)        1
Fatal(37)         1
Fatal(114)        1
Fatal(73)         1
Name: Injury.Severity, Length: 109, dtype: int64

In [ ]:
#cleaning total fatal injuries and injury severity

In [7]:
df['Injury.Severity'] = df.apply(lambda row: 'Non-Fatal' if row['Injury.Severity'] == 'Fatal' and row['Total.Fatal.Injuries'] == 0 else row['Injury.Severity'], axis=1)
print(df[(df['Injury.Severity'] == 'Fatal') & (df['Total.Fatal.Injuries'] == 0)].shape[0])
#changing fatal entries to non-fatal if there are 0 total fatal injurues and checking that it worked

0


In [8]:
def impute_fatalities(row):
    if pd.isna(row['Total.Fatal.Injuries']) and 'Fatal(' in row['Injury.Severity']:
        return int(row['Injury.Severity'].split('(')[1].split(')')[0])
    return row['Total.Fatal.Injuries']

df['Total.Fatal.Injuries'] = df.apply(impute_fatalities, axis=1)

#imputing the number in parenthesis after Fatal from Injury Severity into Total Fatal Injuries when Total Fatal Injuries is null

In [9]:
df["Total.Fatal.Injuries"].isnull().sum()
#checking how many null values remain in Total Fatal Injuries

11386

In [10]:
def impute_nonfatal(row):
    if row['Injury.Severity'] == 'Non-Fatal' and pd.isna(row['Total.Fatal.Injuries']):
        row['Total.Fatal.Injuries'] = 0
    return row

df = df.apply(impute_nonfatal, axis=1)
#imputing Total Fatal Injuries with 0 when it is null and Injury Severity is non fatal

In [11]:
df["Total.Fatal.Injuries"].isnull().sum()
#checking how many null values remain in Total Fatal Injuries

724

In [12]:
df.isnull().sum()
#checking how many null values remain per column

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     225
Injury.Severity             979
Aircraft.damage            3172
Aircraft.Category         56532
Registration.Number        1275
Make                          0
Model                         0
Amateur.Built                99
Number.of.Engines          6023
Engine.Type                7006
Purpose.of.flight          6138
Total.Fatal.Injuries        724
Total.Serious.Injuries    12490
Total.Minor.Injuries      11914
Total.Uninjured            5897
Weather.Condition          4439
Broad.phase.of.flight     27094
Report.Status              6335
Publication.Date          15224
dtype: int64

In [13]:
df[df['Total.Fatal.Injuries'].isnull()]['Injury.Severity'].value_counts()
#checking the value counts in injury severity for rows that are null in total fatal injuries

Incident       640
Unavailable     84
Name: Injury.Severity, dtype: int64

In [14]:
df[df['Injury.Severity'] == 'Unavailable']['Total.Fatal.Injuries'].value_counts(dropna=False)
#checking the value counts for total fatal injuries in rows where injury severity is unavilaible  

NaN    84
0.0    11
Name: Total.Fatal.Injuries, dtype: int64

In [15]:
df[df['Injury.Severity'].isnull()]['Total.Fatal.Injuries'].value_counts()
#checking value counts in total fatal injuries for rows that are null in injury severity

0.0    979
Name: Total.Fatal.Injuries, dtype: int64

In [16]:
def impute_injury_severity(row):
    return 'Non-Fatal' if pd.isnull(row['Injury.Severity']) and row['Total.Fatal.Injuries'] == 0.0 else row['Injury.Severity']

df['Injury.Severity'] = df.apply(impute_injury_severity, axis=1)
#imputing Non-fatal for rows that are null in injury severity but are 0 in total fatal injuries

In [17]:
unavailable_and_null = df[(df['Injury.Severity'] == 'Unavailable') & (df['Total.Fatal.Injuries'].isnull())]
df.drop(unavailable_and_null.index, inplace=True)
#dropping rows that are unavailable for injury severity and null for total fatal injuries

In [18]:
df[df['Total.Fatal.Injuries'].isnull()]['Injury.Severity'].value_counts()
#double checking the value counts in injury severity for rows that are null in total fatal injuries

Incident    640
Name: Injury.Severity, dtype: int64

In [19]:
df = df[~((df['Injury.Severity'] == 'Incident') & df['Total.Fatal.Injuries'].isnull())]
#dropping the last 640 rows that are null in total fatal injuries 

In [ ]:
#finished cleaning total fatal injuries and injury severity

In [20]:
df.isnull().sum()
#checking again for total null entries per column

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     44
Country                     222
Injury.Severity               0
Aircraft.damage            2897
Aircraft.Category         55960
Registration.Number         991
Make                          0
Model                         0
Amateur.Built                46
Number.of.Engines          5702
Engine.Type                6754
Purpose.of.flight          5684
Total.Fatal.Injuries          0
Total.Serious.Injuries    11769
Total.Minor.Injuries      11215
Total.Uninjured            5676
Weather.Condition          4248
Broad.phase.of.flight     26760
Report.Status              6335
Publication.Date          15143
dtype: int64

In [ ]:
#cleaning aircraft damage

In [21]:
df["Aircraft.damage"].value_counts()
#checking value counts for aircraft damage

Substantial    64027
Destroyed      18570
Minor           2440
Unknown          119
Name: Aircraft.damage, dtype: int64

In [22]:
df = df[df['Aircraft.damage'] != 'Unknown']
#removing rows where aircraft damage is unknown

In [23]:
df.dropna(subset=['Aircraft.damage'], inplace=True)
#dropping rows where aircraft damage is null

In [ ]:
#finished cleaning aircraft damage

In [24]:
df.isnull().sum()
#checking again for total null values per column

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     42
Country                     208
Injury.Severity               0
Aircraft.damage               0
Aircraft.Category         54472
Registration.Number         950
Make                          0
Model                         0
Amateur.Built                46
Number.of.Engines          4633
Engine.Type                5568
Purpose.of.flight          4210
Total.Fatal.Injuries          0
Total.Serious.Injuries    11719
Total.Minor.Injuries      11021
Total.Uninjured            5651
Weather.Condition          3126
Broad.phase.of.flight     25048
Report.Status              5189
Publication.Date          14507
dtype: int64

In [ ]:
#cleaning and imputing number of engines

In [25]:
df["Number.of.Engines"].value_counts()
#checking the value counts for number of engines

1.0    69047
2.0     9686
0.0     1060
3.0      308
4.0      300
8.0        2
6.0        1
Name: Number.of.Engines, dtype: int64

In [26]:
#iterating through rows of df where number of engines is null, checking if there is a unique engine number associated with this make/model combo and if so imputing it
for index, row in df[df['Number.of.Engines'].isnull()].iterrows():
    same_make_model = df[(df['Make'] == row['Make']) & (df['Model'] == row['Model'])]
    unique_engines = same_make_model['Number.of.Engines'].dropna().unique()
    if len(unique_engines) == 1: 
        df.at[index, 'Number.of.Engines'] = unique_engines[0]

In [27]:
df.isnull().sum()
#checking again for total null entries per column

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     42
Country                     208
Injury.Severity               0
Aircraft.damage               0
Aircraft.Category         54472
Registration.Number         950
Make                          0
Model                         0
Amateur.Built                46
Number.of.Engines          1839
Engine.Type                5568
Purpose.of.flight          4210
Total.Fatal.Injuries          0
Total.Serious.Injuries    11719
Total.Minor.Injuries      11021
Total.Uninjured            5651
Weather.Condition          3126
Broad.phase.of.flight     25048
Report.Status              5189
Publication.Date          14507
dtype: int64

In [28]:
# Dropping rows where 'Number.of.Engines' is null
df = df.dropna(subset=['Number.of.Engines'])

In [ ]:
#finished cleaning number of engines

In [29]:
#checking for null entries per column again
df.isnull().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     35
Country                     207
Injury.Severity               0
Aircraft.damage               0
Aircraft.Category         53595
Registration.Number         515
Make                          0
Model                         0
Amateur.Built                33
Number.of.Engines             0
Engine.Type                4503
Purpose.of.flight          3549
Total.Fatal.Injuries          0
Total.Serious.Injuries    11367
Total.Minor.Injuries      10635
Total.Uninjured            5338
Weather.Condition          2519
Broad.phase.of.flight     23540
Report.Status              4594
Publication.Date          14255
dtype: int64

In [ ]:
#cleaning and imputing engine type

In [30]:
df["Engine.Type"].value_counts()
#checking engine type value counts

Reciprocating      68900
Turbo Shaft         3476
Turbo Prop          3098
Turbo Fan           1456
Unknown             1223
Turbo Jet            515
None                  13
Electric               8
NONE                   2
Hybrid Rocket          1
LR                     1
UNK                    1
Geared Turbofan        1
Name: Engine.Type, dtype: int64

In [31]:
#checking for unique engine type of make/model combos when engine type is null and imputing it if there is only 1
missing_engine_type = df[df['Engine.Type'].isnull()]

for index, row in missing_engine_type.iterrows():
    same_model = df[(df['Make'] == row['Make']) & (df['Model'] == row['Model']) & (~df['Engine.Type'].isnull())]

    if len(same_model['Engine.Type'].unique()) == 1:
        df.at[index, 'Engine.Type'] = same_model['Engine.Type'].iloc[0]

In [32]:
df.isnull().sum()
#checking again for null values in each column

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     35
Country                     207
Injury.Severity               0
Aircraft.damage               0
Aircraft.Category         53595
Registration.Number         515
Make                          0
Model                         0
Amateur.Built                33
Number.of.Engines             0
Engine.Type                1506
Purpose.of.flight          3549
Total.Fatal.Injuries          0
Total.Serious.Injuries    11367
Total.Minor.Injuries      10635
Total.Uninjured            5338
Weather.Condition          2519
Broad.phase.of.flight     23540
Report.Status              4594
Publication.Date          14255
dtype: int64

In [33]:
#checking for unique engine type of makes when engine type is null and imputing it if there is only 1
missing_engine_type = df[df['Engine.Type'].isnull()]

for index, row in missing_engine_type.iterrows():
    same_make = df[(df['Make'] == row['Make']) & (~df['Engine.Type'].isnull())]

    if len(same_make['Engine.Type'].unique()) == 1:
        df.at[index, 'Engine.Type'] = same_make['Engine.Type'].iloc[0]

In [34]:
df.isnull().sum()
#checking again for null values in each column

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     35
Country                     207
Injury.Severity               0
Aircraft.damage               0
Aircraft.Category         53595
Registration.Number         515
Make                          0
Model                         0
Amateur.Built                33
Number.of.Engines             0
Engine.Type                1246
Purpose.of.flight          3549
Total.Fatal.Injuries          0
Total.Serious.Injuries    11367
Total.Minor.Injuries      10635
Total.Uninjured            5338
Weather.Condition          2519
Broad.phase.of.flight     23540
Report.Status              4594
Publication.Date          14255
dtype: int64

In [35]:
df.dropna(subset=['Engine.Type'], inplace=True)
#dropping all rows that are still null for engine type

In [ ]:
#finished cleaning and imputing engine type

In [36]:
df.isnull().sum()
#checking again for null entries across columns

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     35
Country                     207
Injury.Severity               0
Aircraft.damage               0
Aircraft.Category         53373
Registration.Number         349
Make                          0
Model                         0
Amateur.Built                25
Number.of.Engines             0
Engine.Type                   0
Purpose.of.flight          3234
Total.Fatal.Injuries          0
Total.Serious.Injuries    11146
Total.Minor.Injuries      10382
Total.Uninjured            5150
Weather.Condition          2155
Broad.phase.of.flight     22417
Report.Status              3906
Publication.Date          14087
dtype: int64

In [39]:
#grouping the data by investigation type and checking the avg total fatalities per type
average_fatalities_by_type = df.groupby('Investigation.Type')['Total.Fatal.Injuries'].mean().reset_index()

print(average_fatalities_by_type)

  Investigation.Type  Total.Fatal.Injuries
0           Accident              0.500535
1           Incident              0.001900


In [42]:
#Creating a numerical mapping of damage severity where minor = 1, substantial = 2, and destroyed = 3 and printing the mapping per engine type
damage_severity_mapping = {'Minor': 1, 'Substantial': 2, 'Destroyed': 3}

df['Damage_Severity'] = df['Aircraft.damage'].map(damage_severity_mapping)

avg_damage_by_engine = df.groupby('Engine.Type')['Damage_Severity'].mean().reset_index().sort_values(by='Damage_Severity', ascending=True)
print(avg_damage_by_engine.head())  


       Engine.Type  Damage_Severity
1  Geared Turbofan         1.000000
7        Turbo Fan         1.719486
8        Turbo Jet         1.996198
0         Electric         2.000000
3               LR         2.000000


In [71]:
df["Engine.Type"].value_counts()
#checking engine type value counts

Reciprocating      71410
Turbo Shaft         3762
Turbo Prop          3335
Turbo Fan           1897
Turbo Jet            530
Electric              11
Hybrid Rocket          1
Lr                     1
Geared Turbofan        1
Name: Engine.Type, dtype: int64

In [ ]:
#realizing i forgot certain things when cleaning engine type and going back to do it as well as imputing

In [72]:
df['Engine.Type'] = df['Engine.Type'].str.title().str.strip()
#cleaning engine type by formatting them the same, realizing I forgot to do it earlier

In [73]:
#imputing correct engine types for ambiguous engine type entries if they are one of a kind for that model, if a unique engine type is not found row is deleted
ambiguous_engine_types = ['None', 'Unknown', 'Unk']

ambiguous_df = df[df['Engine.Type'].isin(ambiguous_engine_types)]

imputable_models = df[~df['Engine.Type'].isin(ambiguous_engine_types)].groupby('Model')['Engine.Type'].nunique()
imputable_models = imputable_models[imputable_models == 1].index.tolist()

for model in imputable_models:
    engine_type = df[(df['Model'] == model) & (~df['Engine.Type'].isin(ambiguous_engine_types))]['Engine.Type'].iloc[0]
    df.loc[(df['Model'] == model) & (df['Engine.Type'].isin(ambiguous_engine_types)), 'Engine.Type'] = engine_type

df = df[~df['Engine.Type'].isin(ambiguous_engine_types)]


In [74]:
#checking value counts for engine types
df["Engine.Type"].value_counts()

Reciprocating      71410
Turbo Shaft         3762
Turbo Prop          3335
Turbo Fan           1897
Turbo Jet            530
Electric              11
Hybrid Rocket          1
Lr                     1
Geared Turbofan        1
Name: Engine.Type, dtype: int64

In [ ]:
#beginning analysis by ranking engine number, engine type, and specific models via a composite safety score derived from the combination of avg fatalities and avg damage severity

In [95]:
#creating composite safety score for engine types occuring atleast 10 times based on the mean of avg fatality and avg damage severity and displaying safest engine types
engine_type_counts = df['Engine.Type'].value_counts()
engine_types_with_at_least_10 = engine_type_counts[engine_type_counts >= 10].index

engine_type_data = df[df['Engine.Type'].isin(engine_types_with_at_least_10)]
engine_type_safety_scores = engine_type_data.groupby('Engine.Type').agg(
    Avg_Fatality=('Total.Fatal.Injuries', 'mean'),
    Avg_Damage_Severity=('Damage_Severity', 'mean')
).reset_index()

engine_type_safety_scores['Composite_Safety_Score'] = (engine_type_safety_scores['Avg_Fatality'] + engine_type_safety_scores['Avg_Damage_Severity']) / 2

safest_engine_types = engine_type_safety_scores.sort_values(by='Composite_Safety_Score')

print(safest_engine_types.head())


     Engine.Type  Avg_Fatality  Avg_Damage_Severity  Composite_Safety_Score
0       Electric      0.090909             2.000000                1.045455
1  Reciprocating      0.355945             2.199706                1.277825
5    Turbo Shaft      0.532430             2.232323                1.382376
4     Turbo Prop      0.906147             2.182009                1.544078
3      Turbo Jet      1.545283             2.001887                1.773585


In [96]:
#creating composite safety score for engine numbers occuring atleast 10 times based on the mean of avg fatality and avg damage severity and displaying safest engine numbers
engine_number_counts = df['Number.of.Engines'].value_counts()
engine_numbers_with_at_least_10 = engine_number_counts[engine_number_counts >= 10].index

engine_number_data = df[df['Number.of.Engines'].isin(engine_numbers_with_at_least_10)]
engine_number_safety_scores = engine_number_data.groupby('Number.of.Engines').agg(
    Avg_Fatality=('Total.Fatal.Injuries', 'mean'),
    Avg_Damage_Severity=('Damage_Severity', 'mean')
).reset_index()

engine_number_safety_scores['Composite_Safety_Score'] = (engine_number_safety_scores['Avg_Fatality'] + engine_number_safety_scores['Avg_Damage_Severity']) / 2

safest_engine_numbers = engine_number_safety_scores.sort_values(by='Composite_Safety_Score')

print(safest_engine_numbers.head())


   Number.of.Engines  Avg_Fatality  Avg_Damage_Severity  \
1                1.0      0.330055             2.193212   
0                0.0      0.456929             2.123596   
2                2.0      1.277341             2.190106   
3                3.0      2.722222             1.524691   
4                4.0      5.711656             1.723926   

   Composite_Safety_Score  
1                1.261634  
0                1.290262  
2                1.733723  
3                2.123457  
4                3.717791  


In [94]:
#creating composite safety score for models occuring atleast 10 times based on the mean of avg fatality and avg damage severity and displaying safest models
model_counts = df['Model'].value_counts()
models_with_at_least_10 = model_counts[model_counts >= 10].index

model_data = df[df['Model'].isin(models_with_at_least_10)]
model_safety_scores = model_data.groupby('Model').agg(
    Avg_Fatality=('Total.Fatal.Injuries', 'mean'),
    Avg_Damage_Severity=('Damage_Severity', 'mean')
).reset_index()

model_safety_scores['Composite_Safety_Score'] = (model_safety_scores['Avg_Fatality'] + model_safety_scores['Avg_Damage_Severity']) / 2

safest_models = model_safety_scores.sort_values(by='Composite_Safety_Score')

print(safest_models.head(5))


       Model  Avg_Fatality  Avg_Damage_Severity  Composite_Safety_Score
316  727-223      0.000000             1.076923                0.538462
330      787      0.000000             1.200000                0.600000
536  DC-9-51      0.055556             1.166667                0.611111
687    MD-80      0.000000             1.300000                0.650000
315  727-200      0.000000             1.300000                0.650000


In [97]:
#saving the df with the safest models to a CSV file
safest_models.to_csv('safest_models.csv', index=False)

#saving the df with the safest engine types to a CSV file
safest_engine_types.to_csv('safest_engine_types.csv', index=False)

#saving the df with the safest engine numbers to a CSV file
safest_engine_numbers.to_csv('safest_engine_numbers.csv', index=False)


In [98]:
#saving the top 5 entries of the df with the safest models to a CSV file
safest_models.head(5).to_csv('safest_models_top_5.csv', index=False)

In [ ]:
#My analysis has concluded that the aforementioned engine numbers, engine types, and models are the safest investment option for
#an organization looking to enter into the aircraft industry. This analysis is based upon the idea that investment safety is 
#based primarily upon minimized fatal injuries as well as minimized damage to the aircraft. By using these results to inform 
#decisions regarding which airplanes to purchase, individuals seeking to enter into the airspace industry can reduce the risk of
#incidents in which their passenegers are fatally injured or their planes are irrevocably damaged.
